## Exploratory Data Analysis

In [1]:
# --- Import necessary libraries ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import ipywidgets as widgets
from ipywidgets import interact

# --- Set global Matplotlib parameters ---
plt.rcParams["font.family"] = "serif"
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 18
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 5.0
plt.rcParams["xtick.minor.size"] = 3.0
plt.rcParams["ytick.major.size"] = 5.0
plt.rcParams["ytick.minor.size"] = 3.0
plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["legend.handlelength"] = 2.0

In [2]:
# --- Load training data (Years 1 and 2) ---
train_df = pd.read_excel('.././datasets/training.xlsx', sheet_name='Data')

# --- Load testing data (Year 3)
test_df = pd.read_excel('.././datasets/testing.xlsx', sheet_name='Data')

### Descriptive Statistics

In [3]:
# --- Dynamically find the columns for temperature and GHI ---
load_col = ['Load']
temp_cols = sorted([col for col in train_df.columns if 'Temp' in col])
ghi_cols = sorted([col for col in train_df.columns if 'GHI' in col])
all_cols = load_col + temp_cols + ghi_cols

# --- Create a list to hold the statistics for each year ---
stats_list = []

# --- Loop through each year in the training data ---
for year in sorted(train_df['Year'].unique()):
    # Filter data for the current year
    year_df = train_df[train_df['Year'] == year][all_cols]
    
    # Calculate descriptive statistics
    stats = {
        'Mean': year_df.mean(),
        'Variance': year_df.var(),
        'Median': year_df.median(),
        'Min': year_df.min(),
        'Max': year_df.max(),
        'Skewness': year_df.skew(),
        'Kurtosis': year_df.kurt()
    }
    
    # Convert dictionary to DataFrame and set index
    stats_df = pd.DataFrame(stats).T
    stats_df['Year'] = year
    stats_df = stats_df.set_index('Year', append=True).reorder_levels([1, 0])
    stats_list.append(stats_df)

# --- Concatenate the results for all years into a single DataFrame ---
summary_stats = pd.concat(stats_list)

# --- Create the multi-level column headers for the final table ---
summary_stats.columns = pd.MultiIndex.from_tuples([
    ('Load', ''),
    ('Temperature', 'Site 1'), ('Temperature', 'Site 2'), ('Temperature', 'Site 3'), 
    ('Temperature', 'Site 4'), ('Temperature', 'Site 5'),
    ('Global Horizontal Irradiance', 'Site 1'), ('Global Horizontal Irradiance', 'Site 2'), 
    ('Global Horizontal Irradiance', 'Site 3'), ('Global Horizontal Irradiance', 'Site 4'), 
    ('Global Horizontal Irradiance', 'Site 5')
])

print("Descriptive Statistics for Training Data (Years 1 & 2)")
display(summary_stats.style.format('{:.2f}'))

Descriptive Statistics for Training Data (Years 1 & 2)


### Interactive Plots

#### Daily Variation Analysis

In [4]:
def plot_daily_variation_ghi(month, site):
    fig, axes = plt.subplots(1, 2, figsize=(18, 9), sharey=True)
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        filtered_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        for day in sorted(filtered_df["Day"].unique()):
            subset = filtered_df[filtered_df["Day"] == day]
            ax.plot(subset["Hour"], subset[site], alpha=0.5, linestyle="-", marker="o", lw=1.0)
        centroid = filtered_df.groupby("Hour")[site].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, marker='D', linestyle='-', label="Centroid")
        ax.set_xlabel("Hour of the Day")
        ax.set_title(f"Hourly GHI Variation for {calendar.month_name[month]} (Year {year}) - {site}")
        ax.legend(loc="upper left")
    axes[0].set_ylabel("GHI (W/m$^2$)")
    plt.tight_layout()
    plt.show()

month_widget_ghi = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")
site_columns_ghi = [col for col in train_df.columns if "GHI" in col]
site_widget_ghi = widgets.Dropdown(options=site_columns_ghi, value=site_columns_ghi[0], description="Site:")
interact(plot_daily_variation_ghi, month=month_widget_ghi, site=site_widget_ghi)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_daily_variation_ghi(month, site)>

In [5]:
def plot_daily_variation_temp(month, site):
    fig, axes = plt.subplots(1, 2, figsize=(16, 8), sharey=True)
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        filtered_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        for day in sorted(filtered_df["Day"].unique()):
            subset = filtered_df[filtered_df["Day"] == day]
            ax.plot(subset["Hour"], subset[site], alpha=0.5, linestyle="-", marker="o", lw=1.0)
        centroid = filtered_df.groupby("Hour")[site].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, marker='D', linestyle='-', label="Centroid")
        ax.set_xlabel("Hour of the Day")
        ax.set_title(f"Hourly Temp for {calendar.month_name[month]} (Year {year}) - {site}")
        ax.legend(loc="upper left")
    axes[0].set_ylabel("Temperature (°C)")
    plt.tight_layout()
    plt.show()

month_widget_temp = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")
site_columns_temp = [col for col in train_df.columns if "Temp" in col]
site_widget_temp = widgets.Dropdown(options=site_columns_temp, value=site_columns_temp[0], description="Site:")
interact(plot_daily_variation_temp, month=month_widget_temp, site=site_widget_temp)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_daily_variation_temp(month, site)>

In [6]:
def plot_daily_load_variation(month):
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        filtered_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        for day in sorted(filtered_df["Day"].unique()):
            subset = filtered_df[filtered_df["Day"] == day]
            ax.plot(subset["Hour"], subset["Load"], alpha=0.5, linestyle="-", marker="o", lw=1.0)
        centroid = filtered_df.groupby("Hour")["Load"].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, marker='D', linestyle='-', label="Centroid")
        ax.set_xlabel("Hour of the Day")
        ax.set_title(f"Hourly Load for {calendar.month_name[month]} (Year {year})")
        ax.legend(loc="upper left")
    axes[0].set_ylabel("Electricity Load")
    plt.suptitle(f"Hourly Electricity Load Variation for {calendar.month_name[month]}")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

month_widget_load = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")
interact(plot_daily_load_variation, month=month_widget_load)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_daily_load_variation(month)>

#### Weekly Variation Analysis

In [7]:
# --- Create mappings for month-day calculations ---
days_in_month = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}
cumulative_days_map = {i: sum(list(days_in_month.values())[:i-1]) for i in range(1, 13)}

def add_week_info(df):
    df = df.copy()
    
    df.dropna(subset=['Month', 'Day'], inplace=True)
    df['Month'] = df['Month'].astype(int)
    df['Day'] = df['Day'].astype(int)

    max_days = df['Month'].map(days_in_month)
    df['Day_Clipped'] = np.minimum(df['Day'], max_days)
    cumulative_days = df['Month'].map(cumulative_days_map)
    df['DayOfYear'] = cumulative_days + df['Day_Clipped']

    df['WeekOfYear'] = ((df['DayOfYear'] - 1) // 7) + 1
    df['DayOfWeek'] = (df['DayOfYear'] - 1) % 7
    
    return df

# --- Prepare the training dataframe ---
train_df = add_week_info(train_df)
print("Weekly information has been added to the dataframe successfully.")

Weekly information has been added to the dataframe successfully.


In [8]:
def plot_weekly_comparison(month, site):
    fig, axes = plt.subplots(1, 2, figsize=(18, 8), sharey=True)
    
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        
        # Filter for the selected year and month
        month_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        
        # Create the 'HourOfWeek' column for a continuous x-axis
        month_df = month_df.copy()
        month_df['HourOfWeek'] = month_df['DayOfWeek'] * 24 + month_df['Hour']
        
        # Find all the unique weeks within that month
        weeks_in_month = month_df['WeekOfYear'].unique()
        
        # Plot each individual week's pattern with transparency
        for week in weeks_in_month:
            week_subset = month_df[month_df['WeekOfYear'] == week]
            ax.plot(week_subset['HourOfWeek'], week_subset[site], alpha=0.3, linestyle="-")
            
        # Calculate and plot the centroid (average weekly pattern for the month)
        centroid = month_df.groupby('HourOfWeek')[site].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, linestyle='-', label="Monthly Centroid")
        
        # --- Formatting the plot ---
        ax.set_xlabel("Hour of the Week")
        ax.set_title(f"Weekly GHI Variation in {calendar.month_name[month]} (Year {year}) - {site}")
        ax.legend(loc="upper left")
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # Set x-ticks to correspond to the start of each day for readability
        tick_locations = [i * 24 for i in range(7)]
        tick_labels = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(tick_labels)
        ax.set_xlim(0, 168)

    axes[0].set_ylabel("GHI (W/m$^2$)")
    plt.tight_layout()
    plt.show()

# --- Widgets (reusing your original ones) ---
month_widget_ghi = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")
site_columns_ghi = [col for col in train_df.columns if "GHI" in col]
site_widget_ghi = widgets.Dropdown(options=site_columns_ghi, value=site_columns_ghi[0], description="Site:")

interact(plot_weekly_comparison, month=month_widget_ghi, site=site_widget_ghi)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_weekly_comparison(month, site)>

In [9]:
def plot_weekly_temp_comparison(month, site):
    fig, axes = plt.subplots(1, 2, figsize=(18, 8), sharey=True)
    
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        
        # Filter for the selected year and month
        month_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        
        # Create the 'HourOfWeek' column for a continuous x-axis
        month_df = month_df.copy()
        month_df['HourOfWeek'] = month_df['DayOfWeek'] * 24 + month_df['Hour']
        
        # Find all the unique weeks within that month
        weeks_in_month = month_df['WeekOfYear'].unique()
        
        # Plot each individual week's pattern with transparency
        for week in weeks_in_month:
            week_subset = month_df[month_df['WeekOfYear'] == week]
            ax.plot(week_subset['HourOfWeek'], week_subset[site], alpha=0.3, linestyle="-")
            
        # Calculate and plot the centroid (average weekly pattern for the month)
        centroid = month_df.groupby('HourOfWeek')[site].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, linestyle='-', label="Monthly Centroid")
        
        # --- Formatting the plot ---
        ax.set_xlabel("Hour of the Week")
        ax.set_title(f"Weekly Temperature Variation in {calendar.month_name[month]} (Year {year}) - {site}")
        ax.legend(loc="upper left")
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # Set x-ticks to correspond to the start of each day for readability
        tick_locations = [i * 24 for i in range(7)]
        tick_labels = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(tick_labels)
        ax.set_xlim(0, 168)

    axes[0].set_ylabel("Temperature (°C)")
    plt.tight_layout()
    plt.show()

# --- Widgets for the Temperature plot ---
month_widget_temp = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")
site_columns_temp = [col for col in train_df.columns if "Temp" in col]
site_widget_temp = widgets.Dropdown(options=site_columns_temp, value=site_columns_temp[0], description="Site:")

interact(plot_weekly_temp_comparison, month=month_widget_temp, site=site_widget_temp)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_weekly_temp_comparison(month, site)>

In [10]:
def plot_weekly_load_comparison(month):
    fig, axes = plt.subplots(1, 2, figsize=(18, 8), sharey=True)
    
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        
        # Filter for the selected year and month
        month_df = train_df[(train_df["Year"] == year) & (train_df["Month"] == month)]
        
        # Create the 'HourOfWeek' column for a continuous x-axis
        month_df = month_df.copy()
        month_df['HourOfWeek'] = month_df['DayOfWeek'] * 24 + month_df['Hour']
        
        # Find all the unique weeks within that month
        weeks_in_month = month_df['WeekOfYear'].unique()
        
        # Plot each individual week's pattern with transparency
        for week in weeks_in_month:
            week_subset = month_df[month_df['WeekOfYear'] == week]
            ax.plot(week_subset['HourOfWeek'], week_subset['Load'], alpha=0.3, linestyle="-")
            
        # Calculate and plot the centroid (average weekly pattern for the month)
        centroid = month_df.groupby('HourOfWeek')['Load'].mean()
        ax.plot(centroid.index, centroid, color='black', lw=2.5, linestyle='-', label="Monthly Centroid")
        
        # --- Formatting the plot ---
        ax.set_xlabel("Hour of the Week")
        ax.set_title(f"Weekly Load Variation in {calendar.month_name[month]} (Year {year})")
        ax.legend(loc="upper left")
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # Set x-ticks to correspond to the start of each day for readability
        tick_locations = [i * 24 for i in range(7)]
        tick_labels = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(tick_labels)
        ax.set_xlim(0, 168)

    axes[0].set_ylabel("Electricity Load")
    plt.tight_layout()
    plt.show()

# --- Widgets for the Load plot (only month is needed) ---
month_widget_load = widgets.Dropdown(options={calendar.month_name[m]: m for m in sorted(train_df["Month"].unique()) if m != 0}, value=4, description="Month:")

interact(plot_weekly_load_comparison, month=month_widget_load)

interactive(children=(Dropdown(description='Month:', index=3, options={'January': np.int64(1), 'February': np.…

<function __main__.plot_weekly_load_comparison(month)>

#### Seasonal Variation Analysis

In [11]:
# --- Define Seasons for California (Northern Hemisphere Meteorological) ---
season_map = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
}

# Add the 'Season' column to the dataframe by mapping the 'Month'
train_df['Season'] = train_df['Month'].map(season_map)

print("Season information has been added to the dataframe.")

Season information has been added to the dataframe.


In [15]:
def plot_seasonal_weekly_comparison(season, site, variable_name, y_label):
    fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c'] 
    
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        
        # Filter for the selected year and season
        season_df = train_df[(train_df["Year"] == year) & (train_df["Season"] == season)].copy()
        
        # --- FIX: Create the 'HourOfWeek' column which was missing ---
        season_df['HourOfWeek'] = season_df['DayOfWeek'] * 24 + season_df['Hour']
        
        months_in_season = sorted(season_df['Month'].unique())
        
        for i, month in enumerate(months_in_season):
            month_subset = season_df[season_df['Month'] == month]
            centroid = month_subset.groupby('HourOfWeek')[site].mean()
            ax.plot(centroid.index, centroid, label=calendar.month_name[month], color=colors[i])

        # --- Formatting the plot for 168 hours ---
        ax.set_xlabel("Hour of the Week")
        ax.set_title(f"Average Weekly {variable_name} in {season} (Year {year}) - {site}")
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
        
        tick_locations = [i * 24 for i in range(7)]
        tick_labels = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(tick_labels)
        ax.set_xlim(0, 168)

    axes[0].set_ylabel(f"Average {y_label}")
    plt.tight_layout()
    plt.show()

def plot_seasonal_weekly_load_comparison(season):
    fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    for idx, year in enumerate([1, 2]):
        ax = axes[idx]
        season_df = train_df[(train_df["Year"] == year) & (train_df["Season"] == season)].copy()
        
        # --- FIX: Create the 'HourOfWeek' column which was missing ---
        season_df['HourOfWeek'] = season_df['DayOfWeek'] * 24 + season_df['Hour']

        months_in_season = sorted(season_df['Month'].unique())
        for i, month in enumerate(months_in_season):
            month_subset = season_df[season_df['Month'] == month]
            centroid = month_subset.groupby('HourOfWeek')['Load'].mean()
            ax.plot(centroid.index, centroid, label=calendar.month_name[month], color=colors[i])
            
        ax.set_xlabel("Hour of the Week")
        ax.set_title(f"Average Weekly Load in {season} (Year {year})")
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
        tick_locations = [i * 24 for i in range(7)]
        tick_labels = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(tick_labels)
        ax.set_xlim(0, 168)
        
    axes[0].set_ylabel("Average Electricity Load")
    plt.tight_layout()
    plt.show()

# --- Reusing the Season Widget ---
season_widget = widgets.Dropdown(options=['Winter', 'Spring', 'Summer', 'Autumn'], description="Season:")

# --- Interactive plot for GHI ---
def seasonal_weekly_ghi_plot(season, site):
    plot_seasonal_weekly_comparison(season, site, 'GHI', 'GHI (W/m$^2$)')
interact(seasonal_weekly_ghi_plot, season=season_widget, site=site_widget_ghi)

interactive(children=(Dropdown(description='Season:', options=('Winter', 'Spring', 'Summer', 'Autumn'), value=…

<function __main__.seasonal_weekly_ghi_plot(season, site)>

In [13]:
# --- Interactive plot for Temperature ---
def seasonal_weekly_temp_plot(season, site):
    plot_seasonal_weekly_comparison(season, site, 'Temperature', 'Temperature (°C)')
interact(seasonal_weekly_temp_plot, season=season_widget, site=site_widget_temp)

interactive(children=(Dropdown(description='Season:', options=('Winter', 'Spring', 'Summer', 'Autumn'), value=…

<function __main__.seasonal_weekly_temp_plot(season, site)>

In [14]:
# --- Interactive plot for Load ---
interact(plot_seasonal_weekly_load_comparison, season=season_widget)

interactive(children=(Dropdown(description='Season:', options=('Winter', 'Spring', 'Summer', 'Autumn'), value=…

<function __main__.plot_seasonal_weekly_load_comparison(season)>

### Correlation and Collinearity